### Repaso parcial

#### Librerías

In [ ]:
%pip install scikit-learn
%pip install seaborn
%pip install --upgrade scikit-learn imbalanced-learn
%pip install statsmodels
%pip install mlxtend
%pip install shap

In [ ]:
# Librerías básicas
import numpy as np
import pandas as pd

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, confusion_matrix, classification_report, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.inspection import permutation_importance


# Justicia
from sklearn.preprocessing import StandardScaler
import shap  # Para interpretabilidad


#### Cargar y procesar datos

In [ ]:
# Reemplazar con los datos que te proporcionen
df = pd.read_csv('dataset.csv')

# Dividir en características (X) y etiqueta (y)
X = df.drop('target', axis=1)
y = df['target']

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar si es necesario
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Modelos de regresión

In [ ]:
# Modelo base de regresión
reg_model = RandomForestRegressor(random_state=42)
reg_model.fit(X_train, y_train)

# Predicciones
y_pred = reg_model.predict(X_test)

# Evaluación
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("R2:", r2_score(y_test, y_pred))

# Interpretabilidad
feature_importances = reg_model.feature_importances_
plt.barh(X.columns, feature_importances)
plt.title("Importancia de Características")
plt.show()

# SHAP para explicabilidad
explainer = shap.Explainer(reg_model, X_test)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

# Visualización Local: Predicción individual
shap.force_plot(explainer.expected_value, shap_values[0].values, X[0])

# Visualización de Dependencia
shap.dependence_plot("RM", shap_values.values, X)

El Mean Absolute Error (MAE) mide el promedio de los errores absolutos entre las predicciones del modelo y los valores reales. 

- Escala: Está en las mismas unidades que la variable objetivo.
- Significado: Muestra el error promedio que comete el modelo al hacer predicciones.
- Ejemplo: Si el MAE es 500 y la variable objetivo representa el precio de apartamentos en dólares, significa que, en promedio, el modelo se equivoca por $500 en cada predicción.
- Ventaja: Fácil de interpretar.
- Limitación: No penaliza grandes errores más que pequeños, ya que toma valores absolutos.

El Mean Squared Error (MSE) mide el promedio de los errores elevados al cuadrado

- Escala: Está en las unidades cuadradas de la variable objetivo, lo que dificulta la interpretación directa.
- Significado: Penaliza más los errores grandes que los pequeños debido al cuadrado.
- Ejemplo: Si el MSE es 250,000, significa que los errores al cuadrado suman a esta cantidad en promedio.
- Ventaja: Penaliza grandes desviaciones, útil si el objetivo es evitar errores grandes.
- Limitación: Difícil de interpretar directamente.


El R² mide la proporción de la varianza de la variable dependiente explicada por el modelo

- Escala: Va de 0 a 1 (puede ser negativo si el modelo es peor que simplemente predecir la media).
- Significado: Representa qué porcentaje de la variabilidad de los datos es explicada por el modelo.
- - R² cercano a 1: El modelo explica bien los datos.
- - R² cercano a 0: El modelo no explica la variabilidad de los datos.
- Ejemplo: Un 𝑅2=0.85 indica que el 85% de la variación en los precios de apartamentos es explicada por el modelo.
- Ventaja: Fácil de interpretar como un porcentaje.
- Limitación: No considera el número de variables, lo que puede llevar a problemas de sobreajuste (usa R2 ajustado para corregir esto).

Los valores SHAP (SHapley Additive exPlanations) son una herramienta poderosa para interpretar modelos de machine learning. Se basan en la teoría de juegos y asignan a cada característica un valor que representa su contribución al resultado del modelo.

Interpretación de los Valores SHAP
1. A Nivel Global (Modelo Completo)

Importancia de las características: SHAP puede mostrar las características más importantes al calcular el promedio absoluto de los valores SHAP para cada característica.
- Ejemplo: Si la característica "tamaño" tiene valores SHAP altos en magnitud, significa que el tamaño tiene un gran impacto en las predicciones del modelo.

Visualización Global:

- Summary Plot: Muestra las características más importantes y cómo afectan la predicción.
- Puntos rojos: Valores altos de la característica.
- Puntos azules: Valores bajos de la característica.

2. A Nivel Local (Predicción Individual)

Contribución de cada característica: Los valores SHAP indican cómo cada característica contribuyó a una predicción específica.
- Ejemplo: Para un apartamento específico, "ubicación" podría agregar $20,000 a la predicción, mientras que "tamaño" podría restar $5,000.

Visualización Local:

- Force Plot: Representa gráficamente cómo las características mueven la predicción desde el valor base hacia el valor final.

3. Ejemplo de Resultados

- Summary Plot (Importancia Global):

Muestra las características ordenadas por su importancia.
Puntos dispersos indican cómo los valores altos o bajos de una característica afectan las predicciones.

- Force Plot (Predicción Individual):

Una barra horizontal con el valor base y flechas que representan las contribuciones de las características.
Las flechas hacia la derecha aumentan la predicción; las flechas hacia la izquierda la disminuyen.

- Dependence Plot (Interacciones):

Un gráfico de dispersión que muestra cómo una característica afecta las predicciones y cómo interactúa con otra característica.


#### Modelos de clasificación

In [ ]:
# Modelo base de clasificación
clf_model = RandomForestClassifier(random_state=42)
clf_model.fit(X_train, y_train)

# Predicciones
y_pred = clf_model.predict(X_test)

# Cálculo de métricas
precision = precision_score(y_true, y_pred)  
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Evaluación
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Interpretabilidad
feature_importances = clf_model.feature_importances_
plt.barh(X.columns, feature_importances)
plt.title("Importancia de Características")
plt.show()

# SHAP para explicabilidad
explainer = shap.Explainer(clf_model, X_test)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)


Cómo Interpretar las Métricas
- Precisión (Precision): 

Proporción de verdaderos positivos entre todas las predicciones positivas
Mide qué tan confiable es una predicción positiva.
Ideal para minimizar falsos positivos.

- Recall (Sensibilidad):

Proporción de verdaderos positivos entre todos los casos positivos reales
Indica la capacidad del modelo para identificar positivos.
Ideal para minimizar falsos negativos.

- F1-Score:

Promedio armónico entre precisión y recall
Útil cuando existe un desequilibrio de clases, ya que balancea ambas métricas.

- Informe de Clasificación (Classification Report):

Proporciona un resumen detallado de las métricas por clase y globalmente.

Importante: precisión (que proporción de instancias identifiqué- verdaderas positivas) y recall (cuantas de las que tenía para identificar pude hacerlo correctamente)

Dependiendo el tipo de problema quiero mayor precisión o mayor recall. Ejemplo: Cáncer de pulmón mayor precisión ya que quiero que a las personas que les hago la imagen, la mayoría las identifique correctamente y detección de fraude bancario mayor recall ya que quiero identificar la mayor cantidad de verdaderos fraudes posibles. 

#### Interpretabilidad

1. Cómo se explica la regresión lineal

La regresión lineal es un modelo interpretable basado en una ecuación matemática que describe la relación entre las variables independientes (X) y la variable dependiente (Y)

Interpretación

Cada coeficiente indica la magnitud y dirección de la relación entre la variable independiente y la variable objetivo.
- 𝛽𝑖>0: Relación positiva.
- 𝛽𝑖<0: Relación negativa.

La importancia de las variables puede deducirse directamente del tamaño absoluto de los coeficientes (considerando que las variables estén estandarizadas).

2. Cómo identificar las variables más y menos importantes

Para Modelos Lineales

- Tamaño del Coeficiente (𝛽𝑖): Si las variables están estandarizadas, los coeficientes absolutos más grandes indican mayor importancia.

- Pruebas de Significancia (p-valores): Si un coeficiente tiene un p-valor alto (por ejemplo, > 0.05), puede no ser significativo para explicar 𝑌

Para Modelos No Lineales

- Feature Importance:

En modelos como Random Forest o Gradient Boosting, se utiliza la importancia de características basada en la reducción de error (como Gini o entropía). Librerías como SHAP o LIME permiten medir el impacto de cada variable en la predicción.

Visualizaciones:

- SHAP Summary Plot: Muestra qué variables tienen mayor impacto global.
- Dependence Plot: Muestra cómo una variable afecta las predicciones y si interactúa con otras.

3. Qué es un Random Forest

Un Random Forest es un modelo de machine learning basado en árboles de decisión que utiliza el método de ensamble. Es una colección de árboles de decisión entrenados en diferentes subconjuntos del dataset y características.

Características Clave: Cada árbol hace una predicción y el bosque combina estas predicciones (votación en clasificación o promedio en regresión).

Usa técnicas como:

- Bagging: Selección aleatoria de muestras para entrenar cada árbol.
- Feature Sampling: Selección aleatoria de características para dividir nodos.

Ventajas:

- Robusto frente a overfitting.
- Maneja datos con alta dimensionalidad.
- Permite medir la importancia de las variables.

4. Qué resultados obtengo con un Random Forest

Predicciones:

- En problemas de clasificación, predice la clase con mayor votación.

- En problemas de regresión, predice el promedio de las salidas de los árboles.

Importancia de las Características: Cuantifica qué variables contribuyen más a reducir el error.

Desempeño del Modelo: Métricas como precisión, recall, F1-score (clasificación) o R², MAE, MSE (regresión).

Interpretación Global: Importancia promedio de cada característica.

5. Cómo interpreto el modelo a nivel global y a nivel de instancias

A Nivel Global
- Feature Importance: Muestra el impacto promedio de cada característica en las predicciones.
- Visualizaciones SHAP:
- Distribución de Errores: Analiza si el modelo tiene un desempeño consistente en diferentes rangos del dataset.

A Nivel de Instancias
- SHAP Values:Muestra cómo cada característica contribuye a una predicción específica.
- Force Plot: Representa gráficamente cómo las características mueven la predicción desde el valor base.
- LIME: Genera explicaciones localizadas mediante la perturbación de los datos de entrada.

6. Cómo explico una predicción o grupo de predicciones

Predicción Individual
- SHAP Force Plot:Muestra cómo las características específicas influyen en la predicción. Ejemplo: "El tamaño del apartamento agregó $20,000 y la ubicación restó $5,000 al precio final."
- Análisis de Errores: Compara la predicción con el valor real para detectar patrones o inconsistencias.

Grupo de Predicciones
- SHAP Summary Plot: Destaca patrones globales en la influencia de las características.
- Segmentación: Agrupa instancias con características similares y analiza cómo el modelo predice en esos segmentos.

Explicaciones para Negocio

Presenta insights accionables, como:
- "El precio de venta de apartamentos está fuertemente influenciado por la ubicación y el área."
- "Clientes en estratos altos tienden a obtener precios más precisos."



#### Comparación de modelos

In [ ]:
# Ejemplo: GridSearch para optimizar hiperparámetros
param_grid = {'n_estimators': [100, 200], 'max_depth': [10, 20]}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5)
grid_search.fit(X_train, y_train)

# Validar si mejora es significativa
best_model = grid_search.best_estimator_
y_pred_best = best_model.predict(X_test)

print("Confusion Matrix (Mejor Modelo):\n", confusion_matrix(y_test, y_pred_best))
print("ROC AUC (Mejor Modelo):", roc_auc_score(y_test, best_model.predict_proba(X_test)[:, 1]))


#### Justicia

In [ ]:
# Validar justicia con variables sensibles
sensitive_feature = 'gender'  # Ejemplo: columna sensible
group_a = df[df[sensitive_feature] == 0]
group_b = df[df[sensitive_feature] == 1]

# Comparar métricas entre grupos
for group, data in {'Group A': group_a, 'Group B': group_b}.items():
    y_true = data['target']
    y_pred = clf_model.predict(data.drop('target', axis=1))
    print(f"{group} Metrics")
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))


##### Paridad estadística

In [ ]:
import numpy as np
import pandas as pd

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_pred': [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]   # Predicciones del modelo
})

# Función para calcular la tasa de predicción positiva (paridad estadística)
def positive_rate(data, group_col, pred_col):
    rates = data.groupby(group_col)[pred_col].mean()
    return rates

# Cálculo de tasas de predicción positiva
positive_rates = positive_rate(data, group_col='gender', pred_col='y_pred')

# Mostrar resultados
print("Tasas de predicción positiva por grupo:")
print(positive_rates)

# Validación de paridad estadística
threshold = 0.1  # Diferencia máxima aceptable (ajustable según el caso)
is_fair = np.abs(positive_rates.max() - positive_rates.min()) <= threshold

if is_fair:
    print("\nEl modelo cumple con la paridad estadística.")
else:
    print("\nEl modelo NO cumple con la paridad estadística.")
    print(f"Diferencia entre tasas: {positive_rates.max() - positive_rates.min():.2f}")


##### Paridad condicional estadística

In [ ]:
import numpy as np
import pandas as pd

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'income_level': ['high', 'high', 'medium', 'low', 'low', 'medium', 'low', 'high', 'medium', 'high'],
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_pred': [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]   # Predicciones del modelo
})

# Función para calcular la tasa de predicción positiva condicional
def conditional_positive_rate(data, group_col, condition_col, pred_col):
    """
    Calcula la tasa de predicción positiva condicionada a una característica.
    
    Args:
    - data (DataFrame): Dataset que contiene los datos.
    - group_col (str): Columna del atributo sensible (ej., género).
    - condition_col (str): Columna del atributo condicional (ej., nivel de ingresos).
    - pred_col (str): Columna de las predicciones (ej., y_pred).

    Returns:
    - DataFrame: Tasa de predicción positiva por grupo y condición.
    """
    rates = data.groupby([condition_col, group_col])[pred_col].mean().unstack()
    return rates

# Cálculo de tasas de predicción positiva condicional
conditional_rates = conditional_positive_rate(data, group_col='gender', condition_col='income_level', pred_col='y_pred')

# Mostrar resultados
print("Tasas de predicción positiva condicionadas:")
print(conditional_rates)

# Validación de paridad condicional estadística
threshold = 0.1  # Diferencia máxima aceptable (ajustable según el caso)
max_diff_by_condition = conditional_rates.max(axis=1) - conditional_rates.min(axis=1)
is_fair = (max_diff_by_condition <= threshold).all()

if is_fair:
    print("\nEl modelo cumple con la paridad condicional estadística.")
else:
    print("\nEl modelo NO cumple con la paridad condicional estadística.")
    print(f"Diferencias máximas por condición:\n{max_diff_by_condition}")


##### Paridad de Predicción 

In [ ]:
import numpy as np
import pandas as pd

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_pred': [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]   # Predicciones del modelo
})

# Función para calcular la precisión positiva por grupo (Predictive Parity)
def positive_precision(data, group_col, true_col, pred_col):
    """
    Calcula la precisión positiva por grupo.
    
    Args:
    - data (DataFrame): Dataset con los datos.
    - group_col (str): Columna del atributo sensible (ej., género).
    - true_col (str): Columna de las etiquetas reales (ej., y_true).
    - pred_col (str): Columna de las predicciones del modelo (ej., y_pred).

    Returns:
    - Series: Precisión positiva por grupo.
    """
    group_data = data[data[pred_col] == 1]  # Filtrar solo predicciones positivas
    precision = group_data.groupby(group_col).apply(
        lambda x: np.mean(x[true_col])
    )
    return precision

# Calcular la precisión positiva por grupo
positive_precisions = positive_precision(data, group_col='gender', true_col='y_true', pred_col='y_pred')

# Mostrar resultados
print("Precisión positiva por grupo:")
print(positive_precisions)

# Validación de paridad de predicción
threshold = 0.1  # Diferencia máxima aceptable (ajustable según el caso)
is_fair = np.abs(positive_precisions.max() - positive_precisions.min()) <= threshold

if is_fair:
    print("\nEl modelo cumple con la paridad de predicción.")
else:
    print("\nEl modelo NO cumple con la paridad de predicción.")
    print(f"Diferencia entre precisiones: {positive_precisions.max() - positive_precisions.min():.2f}")


##### Igualdad de precisión total

In [ ]:
import numpy as np
import pandas as pd

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_pred': [1, 0, 1, 1, 1, 0, 0, 1, 0, 1]   # Predicciones del modelo
})

# Función para calcular la precisión total por grupo
def group_accuracy(data, group_col, true_col, pred_col):
    """
    Calcula la precisión total por grupo.
    
    Args:
    - data (DataFrame): Dataset con los datos.
    - group_col (str): Columna del atributo sensible (ej., género).
    - true_col (str): Columna de las etiquetas reales (ej., y_true).
    - pred_col (str): Columna de las predicciones del modelo (ej., y_pred).

    Returns:
    - Series: Precisión total por grupo.
    """
    accuracy = data.groupby(group_col).apply(
        lambda x: np.mean(x[true_col] == x[pred_col])
    )
    return accuracy

# Calcular la precisión total por grupo
accuracy_by_group = group_accuracy(data, group_col='gender', true_col='y_true', pred_col='y_pred')

# Mostrar resultados
print("Precisión total por grupo:")
print(accuracy_by_group)

# Validación de igualdad de precisión total
threshold = 0.1  # Diferencia máxima aceptable (ajustable según el caso)
is_fair = np.abs(accuracy_by_group.max() - accuracy_by_group.min()) <= threshold

if is_fair:
    print("\nEl modelo cumple con la igualdad de precisión total.")
else:
    print("\nEl modelo NO cumple con la igualdad de precisión total.")
    print(f"Diferencia entre precisiones: {accuracy_by_group.max() - accuracy_by_group.min():.2f}")


##### Buena calibración

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import brier_score_loss

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_prob': [0.9, 0.1, 0.8, 0.7, 0.4, 0.3, 0.6, 0.2, 0.5, 0.9]  # Probabilidades predichas
})

# Función para calcular el Brier Score por grupo
def brier_score_by_group(data, group_col, true_col, prob_col):
    """
    Calcula el Brier Score por grupo.

    Args:
    - data (DataFrame): Dataset con los datos.
    - group_col (str): Columna del atributo sensible (ej., género).
    - true_col (str): Columna de las etiquetas reales (ej., y_true).
    - prob_col (str): Columna de las probabilidades predichas (ej., y_prob).

    Returns:
    - Series: Brier Score por grupo.
    """
    return data.groupby(group_col).apply(
        lambda x: brier_score_loss(x[true_col], x[prob_col])
    )

# Calcular el Brier Score por grupo
brier_scores = brier_score_by_group(data, group_col='gender', true_col='y_true', prob_col='y_prob')

# Mostrar resultados
print("Brier Score por grupo:")
print(brier_scores)

# Validación de buena calibración
threshold = 0.05  # Diferencia máxima aceptable (ajustable según el contexto)
is_fair = np.abs(brier_scores.max() - brier_scores.min()) <= threshold

if is_fair:
    print("\nEl modelo cumple con buena calibración.")
else:
    print("\nEl modelo NO cumple con buena calibración.")
    print(f"Diferencia entre Brier Scores: {brier_scores.max() - brier_scores.min():.2f}")


##### Discriminación causal

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from econml.dml import CausalForestDML
from sklearn.metrics import accuracy_score

# Ejemplo de datos
data = pd.DataFrame({
    'gender': ['male', 'female', 'male', 'female', 'female', 'male', 'female', 'male', 'female', 'male'],
    'age': [25, 30, 45, 35, 50, 40, 28, 60, 22, 27],  # Edad como variable no sensible
    'education': [1, 2, 3, 2, 1, 1, 3, 2, 2, 3],  # Nivel educativo
    'y_true': [1, 0, 1, 1, 0, 0, 1, 0, 0, 1],  # Etiquetas reales
    'y_pred': [0.9, 0.1, 0.8, 0.7, 0.4, 0.3, 0.6, 0.2, 0.5, 0.9]  # Probabilidades predichas
})

# Separar las variables predictoras y la variable sensible
X = data[['age', 'education']]  # Variables no sensibles
T = data['gender']  # Atributo sensible (gender)
y = data['y_true']  # Etiqueta

# Entrenar un modelo con y_pred como la predicción
model = LogisticRegression()
X_train, X_test, y_train, y_test, T_train, T_test = train_test_split(X, y, T, test_size=0.3, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f"Precisión del modelo sin considerar el atributo sensible: {accuracy:.2f}")

# Usamos CausalForestDML para analizar la causalidad y la discriminación
cf = CausalForestDML(model_y=LogisticRegression(), model_t=LogisticRegression(), discrete_treatment=True)

cf.fit(X_train, y_train, T_train)

# Análisis de la discriminación causal - Predicción ajustada sin el atributo sensible
y_pred_adjusted = cf.predict(X_test)

# Comparación de las predicciones ajustadas con las originales
print(f"Precisión del modelo con ajuste causal: {accuracy_score(y_test, y_pred_adjusted):.2f}")

# Si la diferencia es significativa, hay discriminación causal
discriminacion_causal = np.abs(accuracy - accuracy_score(y_test, y_pred_adjusted))

threshold = 0.05  # Ajusta este umbral según el contexto
if discriminacion_causal > threshold:
    print("\nEl modelo muestra discriminación causal.")
else:
    print("\nEl modelo no muestra discriminación causal.")


#### Despliegue

In [ ]:
# Guardar modelo para despliegue
import joblib
joblib.dump(best_model, 'best_model.pkl')

# Cargar y predecir con modelo desplegado
model = joblib.load('best_model.pkl')
predictions = model.predict(X_test)


#### Pruebas A/B

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error
from scipy.stats import ttest_rel, wilcoxon

# Paso 1: Entrenar dos modelos
# Modelo A (Base)
model_a = RandomForestClassifier(random_state=42)
model_a.fit(X_train, y_train)
y_pred_a = model_a.predict(X_test)

# Modelo B (Mejorado)
model_b = LogisticRegression(max_iter=1000, random_state=42)
model_b.fit(X_train, y_train)
y_pred_b = model_b.predict(X_test)

# Paso 2: Evaluar los modelos
# (Cambia a las métricas correspondientes si es un problema de regresión)
accuracy_a = accuracy_score(y_test, y_pred_a)
accuracy_b = accuracy_score(y_test, y_pred_b)

print(f"Accuracy Modelo A: {accuracy_a}")
print(f"Accuracy Modelo B: {accuracy_b}")

# Paso 3: Comparar predicciones (Prueba A/B)
# Métrica para cada ejemplo individual
errors_a = y_test != y_pred_a
errors_b = y_test != y_pred_b

# T-Test pareado (diferencias medias)
t_stat, p_value = ttest_rel(errors_a, errors_b)
print(f"T-Test: t-statistic = {t_stat}, p-value = {p_value}")

# Prueba de Wilcoxon (no paramétrica)
wilcoxon_stat, wilcoxon_p = wilcoxon(errors_a, errors_b)
print(f"Wilcoxon: stat = {wilcoxon_stat}, p-value = {wilcoxon_p}")

# Interpretación de resultados
if p_value < 0.05:
    print("La diferencia entre los modelos es estadísticamente significativa (T-Test).")
else:
    print("No hay evidencia suficiente para afirmar que los modelos son diferentes (T-Test).")

if wilcoxon_p < 0.05:
    print("La diferencia entre los modelos es estadísticamente significativa (Wilcoxon).")
else:
    print("No hay evidencia suficiente para afirmar que los modelos son diferentes (Wilcoxon).")

# Paso 4: Visualizar diferencias en desempeño
# Porcentaje de aciertos por modelo
errors_a_mean = np.mean(errors_a)
errors_b_mean = np.mean(errors_b)

plt.bar(['Modelo A', 'Modelo B'], [1-errors_a_mean, 1-errors_b_mean])
plt.ylabel('Proporción de aciertos')
plt.title('Comparación entre modelos')
plt.show()


In [ ]:
# Calcular errores absolutos en regresión
errors_a = np.abs(y_test - model_a.predict(X_test))
errors_b = np.abs(y_test - model_b.predict(X_test))


#### ROI

In [ ]:
# Supuestos de negocio
income_per_correct_prediction = 100  # Ingreso por predicción correcta
cost_per_error = 50  # Costo por predicción incorrecta
model_cost = 10000  # Costo de desarrollo e implementación

# Ingresos y costos del Modelo A
correct_a = sum(y_test == y_pred_a)
incorrect_a = sum(y_test != y_pred_a)
income_a = correct_a * income_per_correct_prediction
cost_a = incorrect_a * cost_per_error + model_cost
roi_a = (income_a - cost_a) / model_cost

# Ingresos y costos del Modelo B
correct_b = sum(y_test == y_pred_b)
incorrect_b = sum(y_test != y_pred_b)
income_b = correct_b * income_per_correct_prediction
cost_b = incorrect_b * cost_per_error + model_cost
roi_b = (income_b - cost_b) / model_cost

# Punto de Equilibrio para el Modelo Mejorado
break_even_correct_predictions = model_cost / income_per_correct_prediction

# Resultados
print(f"Modelo A - Ingreso: ${income_a}, Costo: ${cost_a}, ROI: {roi_a:.2f}")
print(f"Modelo B - Ingreso: ${income_b}, Costo: ${cost_b}, ROI: {roi_b:.2f}")
print(f"Punto de Equilibrio (Modelo Mejorado): {break_even_correct_predictions:.0f} predicciones correctas")

# Comparación Visual
models = ['Modelo A', 'Modelo B']
rois = [roi_a, roi_b]

plt.bar(models, rois)
plt.ylabel('ROI')
plt.title('Comparación de ROI entre Modelos')
plt.axhline(0, color='red', linestyle='--', label='Punto de Equilibrio')
plt.legend()
plt.show()
